In [1]:
import pandas as pd
from tabulate import tabulate
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import math
from collections import Counter
from scipy.integrate import quad
from scipy.stats import pareto
from IPython.display import display, HTML
import os

In [2]:
def calculate_G(graph):
    degrees_list = []
    sum_difference = 0
    for node_index, deg in graph.degree():
        degrees_list.append(deg)
    degrees = np.array(degrees_list)
    N = len(degrees)
    mean_degree = np.mean(degrees)

    for i in range(N):
        for j in range(N):
            sum_difference += abs(degrees[i]-degrees[j])

    G = sum_difference / (2 * N ** 2 * mean_degree)
    return np.round(G, 2)

In [3]:
def c_bar(graph, x):
    degrees = np.sort(np.array([deg for _,deg in graph.degree()]))
    sum_degrees = np.sum(degrees)
    density = density_degree(graph)

    degrees_list = []
    for degree, p in density.items():
        if degree >= x :
            degrees_list.append(p)

    return np.round(np.sum(degrees_list),2)


def sigma_x(graph, x):
    degrees = np.array([deg for _,deg in graph.degree()])
    mean_x = np.mean(degrees)
    densities = density_degree(graph)

    term = 0
    x_int = int(int(x)) 
    for y in range(x_int, max(degrees)+1):
        term += ((y/mean_x)-1)*densities.get(y, 0)
    return term

def calculate_sigma_tot(graph):
    degrees = np.array([deg for _, deg in graph.degree()])

    sigma_x_val = []
    for degree in degrees:
        sigma_x_val.append(sigma_x(graph, degree))

    # c_bar_values = []
    # for degree in degrees:
    #     c_bar_values.append(c_bar(graph, degree))

    # sigma_c_bar_values = []

    # for i in c_bar_values:
    #     sigma_c_bar_values.append(sigma_x(graph, i))
        
    result = np.mean(sigma_x_val)
    return result

def density_degree(graph):
    degrees = np.array([deg for _, deg in graph.degree()])
    degree_count = Counter(degrees)
    total_nodes = len(degrees)
    density = {}
    # probabilities = []
    sum_list = 0
    for degree, count in degree_count.items():
        density[int(degree)] = count/total_nodes
        # probabilities.append(count/total_nodes)
        sum_list += count/total_nodes
    # print(np.round(sum_list,2))
    # print(np.round(np.sum(probabilities)),2)
    return density

In [4]:
def communismpp_graph(n):
    G = nx.Graph()

    rectangle_points = [(0, 0), (1, 0), (1, 1), (0, 1)]
    
    for i, point in enumerate(rectangle_points):
        G.add_node(i, pos=point)

    G.add_edges_from([(0, 2), (1, 3)])
    
    G.add_edges_from([(0, 1), (1, 2), (2, 3), (3, 0)])

    external_nodes = []
    distance = 0.8 
    
    angle_ranges = [
        (-3*np.pi / 5, - 5*np.pi / 5),  
        (-0.2*np.pi / 5, -2*np.pi / 5),
        (-0.2*np.pi / 5, 2*np.pi / 5),
        (3 * np.pi / 5, 5 * np.pi / 5)
    ]

    for i, (x, y) in enumerate(rectangle_points):
        angle_min, angle_max = angle_ranges[i]
        base_angle = (angle_max - angle_min) / n 
        
        for j in range(n):
            angle = angle_min + j * base_angle
            ext_x = x + distance * np.cos(angle)
            ext_y = y + distance * np.sin(angle)
            external_node = len(G.nodes)
            G.add_node(external_node, pos=(ext_x, ext_y))
            external_nodes.append(external_node)
            G.add_edge(i, external_node)
    return G


def create_cycle_graph(n):
    G = nx.cycle_graph(n)

    for i in range(n):
        G.add_edge(i, (i+2) % n)

    return G

In [6]:
def cayley_tree(coord, depth):
    if depth == 0:
        return nx.empty_graph(1)
    return nx.balanced_tree(coord, depth)
    
def prism_graph(n):
    G = nx.Graph()
    base1 = list(range(n))
    base2 = [v + n for v in base1]
    G.add_nodes_from(base1 + base2)
    for i in range(n):
        G.add_edge(i, (i + 1) % n)          
        G.add_edge(i + n, ((i + 1) % n) + n)
        G.add_edge(i, i + n)             
    return G

def antiprism_graph(n):
    G = nx.Graph()
    base_nodes = list(range(n))
    G.add_nodes_from(base_nodes)
    for i in range(n):
        G.add_edge(i, (i + 1) % n)
        G.add_edge(i, (i + 2) % n)
    return G

def supernova(clique, leaf):
    G = nx.complete_graph(clique)
    new_nodes = range(clique, clique + leaf)
    G.add_nodes_from(new_nodes)
    for new_node in new_nodes:
        for clique_node in range(clique):
            G.add_edge(new_node, clique_node)
    return G

def create_agave_graph(n, steps):
    G = nx.Graph()
    G.add_nodes_from(range(n))
    for i in range(steps + 1):
        for j in range(i + 1, n):
            G.add_edge(i, j)
    return G

output_dir = "graph_images"
os.makedirs(output_dir, exist_ok=True)

def save_graph_image(graph, filename):
    filepath = os.path.join(output_dir, filename)
    plt.figure(figsize=(4, 4))
    nx.draw(graph, with_labels=False, node_size=30, node_color="orange", edge_color="gray")
    plt.savefig(filepath)
    plt.close()


graphs = {
    "prism": prism_graph(10),
    "antiprism": antiprism_graph(8),
    "cayley_tree": cayley_tree(3, 4),
    "supernova": supernova(5, 4),
    "agave": create_agave_graph(20, 5),
    "scale_free_graph": nx.scale_free_graph(20),
    "erdos renya": nx.erdos_renyi_graph(100, 0.2),
    "communism ++": communismpp_graph(100),
    "star graph": nx.star_graph(400),
    "complete graph(communism)": nx.complete_graph(10),
    "cycle graph": create_cycle_graph(10)
}


names, sigma_values, G_values, graph_images = [], [], [], []

for name, graph in graphs.items():
    save_graph_image(graph, f"{name}.png")
    sigma_c = calculate_sigma_tot(graph)
    g_value = calculate_G(graph)
    
    names.append(name)
    sigma_values.append(sigma_c)
    G_values.append(g_value)
    graph_images.append(f"<img src='{name}.png' width='100'>")

data = {
    "Names": names,
    "Gintropy": sigma_values,
    "G": G_values,
    "Graph Image": graph_images
}

df = pd.DataFrame(data)
display(HTML(df.to_html(escape=False, index=False)))


Names,Gintropy,G,Graph Image
prism,0.000000,0.00,
antiprism,0.000000,0.00,
cayley_tree,0.108361,0.33,
supernova,0.061728,0.11,
agave,0.082727,0.28,
scale_free_graph,0.207406,0.66,
erdos renya,0.053163,0.10,
communism ++,0.004926,0.50,
star graph,0.001241,0.50,
complete graph(communism),0.000000,0.00,
